In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

In [3]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning_v4\Features\FeaturesSC"

## <span style="color:yellow"> **Preparamos el set de entramiento para sacar features** </span> 

In [4]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning_v4\Training Sets"

In [5]:
auc_18_20 = pd.read_csv( loc_ts + "\\auc_18_20.csv" ); 
auc_21_23 = pd.read_csv( loc_ts + "\\auc_21_23.csv" );  

## <span style="color:yellow"> **Preparamos el ref_hash de cada ventana** </span> 

In [6]:
loc_lb = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning_v4\Labels"

In [7]:
lb_ins_21_23 = pd.read_csv( loc_lb + "\\label_ins_21_23.csv" );
lb_ins_24_26 = pd.read_csv( loc_lb + "\\label_ins_24_26.csv" ); 

In [8]:
rh_trn = lb_ins_21_23[ ["ref_hash"] ]
rh_tst = lb_ins_24_26[ ["ref_hash"] ]

tg_trn = lb_ins_21_23[ ["21_23_sc"] ]
tg_tst = lb_ins_24_26[ ["24_26_sc"] ]

## <span style="color:yellow"> **=======================================================================================================** </span> 

## <span style="color:green"> **Hora mas popular del usuario en las subastas** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. 

In [12]:
main_ahr_18_20 = rh_trn.copy()
main_ahr_21_23 = rh_tst.copy()

hour_mode_18_20 = auc_18_20[["device_id"]]; hour_mode_18_20["hour"] = pd.to_datetime( auc_18_20["date"] ).dt.hour
hour_mode_21_23 = auc_21_23[["device_id"]]; hour_mode_21_23["hour"] = pd.to_datetime( auc_21_23["date"] ).dt.hour

hour_mode_18_20 = hour_mode_18_20.groupby( ["device_id", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()
hour_mode_21_23 = hour_mode_21_23.groupby( ["device_id", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()

hour_mode_18_20 = hour_mode_18_20.sort_values( ["device_id","count"], ascending = False ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id":"ref_hash"} ).drop("count",axis = 1)
hour_mode_21_23 = hour_mode_21_23.sort_values( ["device_id","count"], ascending = False ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id":"ref_hash"} ).drop("count",axis = 1)

main_ahr_18_20 = main_ahr_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )
main_ahr_21_23 = main_ahr_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )

main_ahr_18_20["main_hour"] =  main_ahr_18_20["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
main_ahr_21_23["main_hour"] =  main_ahr_21_23["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

main_ahr_18_20.to_csv( loc_ftr + "\\main_ahr_trn.csv", index = False )
main_ahr_21_23.to_csv( loc_ftr + "\\main_ahr_tst.csv", index = False )

## <span style="color:green"> **Horario de la primer subasta en ese ventana** </span> 

In [13]:
hr_f_auc_18_20 = rh_trn.copy()
hr_f_auc_21_23 = rh_tst.copy()

first_auc_hour_18_20 = auc_18_20[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )
first_auc_hour_21_23 = auc_21_23[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )

first_auc_hour_18_20["time_to_frt_auc"] = ( pd.to_datetime( first_auc_hour_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_auc_hour_21_23["time_to_frt_auc"] = ( pd.to_datetime( first_auc_hour_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_auc_hour_18_20["hour_frt_auc"] = pd.to_datetime( first_auc_hour_18_20["date"] ).dt.hour
first_auc_hour_21_23["hour_frt_auc"] = pd.to_datetime( first_auc_hour_21_23["date"] ).dt.hour

first_auc_hour_18_20.drop( ["date", "time_to_frt_auc"], axis = 1, inplace = True )
first_auc_hour_21_23.drop( ["date", "time_to_frt_auc"], axis = 1, inplace = True )

hr_f_auc_18_20 = hr_f_auc_18_20.merge( first_auc_hour_18_20, how = "left", on = "ref_hash" )
hr_f_auc_21_23 = hr_f_auc_21_23.merge( first_auc_hour_21_23, how = "left", on = "ref_hash" )

hr_f_auc_18_20.to_csv( loc_ftr + "\\hr_f_auc_trn.csv", index = False )
hr_f_auc_21_23.to_csv( loc_ftr + "\\hr_f_auc_tst.csv", index = False )

## <span style="color:green"> **Cantidad de Subastas por dispositivo en la ventana previa a la conversion** </span>

In [14]:
cant_auc_18_20 = rh_trn.copy()
cant_auc_21_23 = rh_tst.copy()

cant_auc_18_20 = cant_auc_18_20.merge( auc_18_20["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_auc_21_23 = cant_auc_21_23.merge( auc_21_23["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_auc_18_20.fillna( 0, inplace = True )
cant_auc_21_23.fillna( 0, inplace = True )

cant_auc_18_20.to_csv( loc_ftr + "\\cant_auc_trn.csv", index = False )
cant_auc_21_23.to_csv( loc_ftr + "\\cant_auc_tst.csv", index = False )

## <span style="color:green"> **Tiempo hasta la primer auction en ese ventana** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer auction

In [15]:
frst_auc_18_20 = rh_trn.copy()
frst_auc_21_23 = rh_tst.copy()

first_auctions_rh_ins_18_20 = auc_18_20[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )
first_auctions_rh_ins_21_23 = auc_21_23[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )

first_auctions_rh_ins_18_20["time_to_frt_auc"] = ( pd.to_datetime( first_auctions_rh_ins_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_auctions_rh_ins_21_23["time_to_frt_auc"] = ( pd.to_datetime( first_auctions_rh_ins_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_auctions_rh_ins_18_20.drop( "date", axis = 1, inplace = True )
first_auctions_rh_ins_21_23.drop( "date", axis = 1, inplace = True )

frst_auc_18_20 = frst_auc_18_20.merge( first_auctions_rh_ins_18_20, how = "left", on = "ref_hash" )
frst_auc_21_23 = frst_auc_21_23.merge( first_auctions_rh_ins_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_auc_18_20.fillna( 3 * 24 * 3600, inplace = True )
frst_auc_21_23.fillna( 3 * 24 * 3600, inplace = True )

frst_auc_18_20.to_csv( loc_ftr + "\\frst_auc_trn.csv", index = False )
frst_auc_21_23.to_csv( loc_ftr + "\\frst_auc_tst.csv", index = False )

## <span style="color:green"> **Source que recibio mas instalaciones por dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la fuente principal de algun dispositivo) - <span style="color:red"> **No Mejora este tipo de codificacion** </span>

In [16]:
srce_auc_18_20 = rh_trn.copy()
srce_auc_21_23 = rh_tst.copy()

main_source_18_20 = auc_18_20.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()
main_source_21_23 = auc_21_23.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()

main_source_18_20 = main_source_18_20.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_18_20["cant_auc"] )
main_source_21_23 = main_source_21_23.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_21_23["cant_auc"] )

srce_auc_18_20 = srce_auc_18_20.merge( main_source_18_20.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_18_20.fillna( "no_font", inplace = True )
srce_auc_21_23 = srce_auc_21_23.merge( main_source_21_23.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_21_23.fillna( "no_font", inplace = True )

srce_auc_18_20["to_count"] = 1; srce_auc_18_20["source_id"] = srce_auc_18_20[["source_id", "to_count"]].groupby("source_id").transform( "sum" ) / len(srce_auc_18_20); del(srce_auc_18_20["to_count"])
srce_auc_21_23["to_count"] = 1; srce_auc_21_23["source_id"] = srce_auc_21_23[["source_id", "to_count"]].groupby("source_id").transform( "sum" ) / len(srce_auc_21_23); del(srce_auc_21_23["to_count"])

srce_auc_18_20.to_csv( loc_ftr + "\\srce_auc_trn.csv", index = False )
srce_auc_21_23.to_csv( loc_ftr + "\\srce_auc_tst.csv", index = False )

#### <span style="color:orange"> **OneHotEncoding** </span> 

In [18]:
srce_auc_18_20 = rh_trn.copy()
srce_auc_21_23 = rh_tst.copy()

main_source_18_20 = auc_18_20.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()
main_source_21_23 = auc_21_23.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()

main_source_18_20 = main_source_18_20.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_18_20["cant_auc"] )
main_source_21_23 = main_source_21_23.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_21_23["cant_auc"] )

srce_auc_18_20 = srce_auc_18_20.merge( main_source_18_20.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_18_20.fillna( "no_font", inplace = True )
srce_auc_21_23 = srce_auc_21_23.merge( main_source_21_23.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_21_23.fillna( "no_font", inplace = True )

#Ya fue controlado que quedan las mismas fuentes para ambas ventanas, no quedan columnas distintas
srce_auc_ohe_18_20 = pd.get_dummies( srce_auc_18_20[ ["source_id"] ].astype( "category" ) )
srce_auc_ohe_21_23 = pd.get_dummies( srce_auc_21_23[ ["source_id"] ].astype( "category" ) )

srce_auc_18_20 = pd.concat( [srce_auc_18_20, srce_auc_ohe_18_20], axis = 1 ); del( srce_auc_18_20["source_id"] )
srce_auc_21_23 = pd.concat( [srce_auc_21_23, srce_auc_ohe_21_23], axis = 1 ); del( srce_auc_21_23["source_id"] )

srce_auc_18_20.to_csv( loc_ftr + "\\srce_auc_trn.csv", index = False )
srce_auc_21_23.to_csv( loc_ftr + "\\srce_auc_tst.csv", index = False )

## <span style="color:green"> **Tiempo hasta la ultima auction en esa ventana** </span> 

In [19]:
last_auc_18_20 = rh_trn.copy()
last_auc_21_23 = rh_tst.copy()

last_auctions_rh_ins_18_20 = auc_18_20[ ["device_id","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )
last_auctions_rh_ins_21_23 = auc_21_23[ ["device_id","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )

last_auctions_rh_ins_18_20["time_to_lst_auc"] = ( pd.to_datetime( last_auctions_rh_ins_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
last_auctions_rh_ins_21_23["time_to_lst_auc"] = ( pd.to_datetime( last_auctions_rh_ins_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

last_auctions_rh_ins_18_20.drop( "date", axis = 1, inplace = True )
last_auctions_rh_ins_21_23.drop( "date", axis = 1, inplace = True )

last_auc_18_20 = last_auc_18_20.merge( last_auctions_rh_ins_18_20, how = "left", on = "ref_hash" )
last_auc_21_23 = last_auc_21_23.merge( last_auctions_rh_ins_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
last_auc_18_20.fillna( 3 * 24 * 3600, inplace = True )
last_auc_21_23.fillna( 3 * 24 * 3600, inplace = True )

last_auc_18_20.to_csv( loc_ftr + "\\last_auc_trn.csv", index = False )
last_auc_21_23.to_csv( loc_ftr + "\\last_auc_tst.csv", index = False )

## <span style="color:green"> **Participo en mas de una subasta** </span> 

#### <span style="color:Orange"> **One-hot Encoding** </span> 

In [9]:
cant_auc_18_20 = rh_trn.copy()
cant_auc_21_23 = rh_tst.copy()

cant_auc_18_20 = cant_auc_18_20.merge( auc_18_20["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "mt_1_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_auc_21_23 = cant_auc_21_23.merge( auc_21_23["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "mt_1_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_auc_18_20.fillna( 0, inplace = True )
cant_auc_21_23.fillna( 0, inplace = True )

cant_auc_18_20['mt_1_ins'] = (cant_auc_18_20['mt_1_ins'] > 1).astype('int8')
cant_auc_21_23['mt_1_ins'] = (cant_auc_21_23['mt_1_ins'] > 1).astype('int8')

cant_auc_18_20.to_csv( loc_ftr + "\\mas1_auc_trn.csv", index = False )
cant_auc_21_23.to_csv( loc_ftr + "\\mas1_auc_tst.csv", index = False )

## <span style="color:green"> **Recibio subasta entre 21 hs y 3 hs (Noche)** </span> 

#### <span style="color:Orange"> **One-hot Encoding** </span> 

In [10]:
auc_18_20['auc_21_3'] = ( pd.to_datetime( auc_18_20['date'] ).dt.hour < 4) | ( pd.to_datetime( auc_18_20['date'] ).dt.hour > 20)
auc_21_23['auc_21_3'] = ( pd.to_datetime( auc_21_23['date'] ).dt.hour < 4) | ( pd.to_datetime( auc_21_23['date'] ).dt.hour > 20)

auc_night_18_20 = rh_trn.copy()
auc_night_21_23 = rh_tst.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_night_18_20 = auc_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_night_21_23 = auc_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_night_18_20["auc_21_3"] =  (auc_night_18_20["auc_21_3"] > 1).astype('int8')
auc_night_21_23["auc_21_3"] =  (auc_night_21_23["auc_21_3"] > 1).astype('int8')

auc_night_18_20.to_csv( loc_ftr + "\\auc_nght_trn.csv", index = False )
auc_night_21_23.to_csv( loc_ftr + "\\auc_nght_tst.csv", index = False )

del auc_18_20['auc_21_3']
del auc_21_23['auc_21_3']

## <span style="color:green"> **Recibio subasta entre 4 hs y 10 hs (Mañana)** </span> 

#### <span style="color:Orange"> **One-hot Encoding** </span> 

In [11]:
auc_18_20['auc_4_10'] = ( pd.to_datetime( auc_18_20['date'] ).dt.hour < 11) & ( pd.to_datetime( auc_18_20['date'] ).dt.hour > 3)
auc_21_23['auc_4_10'] = ( pd.to_datetime( auc_21_23['date'] ).dt.hour < 11) & ( pd.to_datetime( auc_21_23['date'] ).dt.hour > 3)

auc_morn_18_20 = rh_trn.copy()
auc_morn_21_23 = rh_tst.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_4_10':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_4_10':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_morn_18_20 = auc_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_morn_21_23 = auc_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_morn_18_20["auc_4_10"] =  (auc_morn_18_20["auc_4_10"] > 1).astype('int8')
auc_morn_21_23["auc_4_10"] =  (auc_morn_21_23["auc_4_10"] > 1).astype('int8')

auc_morn_18_20.to_csv( loc_ftr + "\\auc_morn_trn.csv", index = False )
auc_morn_21_23.to_csv( loc_ftr + "\\auc_morn_tst.csv", index = False )

del auc_18_20['auc_4_10']
del auc_21_23['auc_4_10']

## <span style="color:green"> **Recibio subasta entre 11 hs y 15 hs (Mediodia)** </span> 

#### <span style="color:Orange"> **One-hot Encoding** </span> 

In [12]:
auc_18_20['auc_11_15'] = ( pd.to_datetime( auc_18_20['date'] ).dt.hour < 16) & ( pd.to_datetime( auc_18_20['date'] ).dt.hour > 10)
auc_21_23['auc_11_15'] = ( pd.to_datetime( auc_21_23['date'] ).dt.hour < 16) & ( pd.to_datetime( auc_21_23['date'] ).dt.hour > 10)

auc_midday_18_20 = rh_trn.copy()
auc_midday_21_23 = rh_tst.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_11_15':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_11_15':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_midday_18_20 = auc_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_midday_21_23 = auc_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_midday_18_20["auc_11_15"] =  (auc_midday_18_20["auc_11_15"] > 1).astype('int8')
auc_midday_21_23["auc_11_15"] =  (auc_midday_21_23["auc_11_15"] > 1).astype('int8')

auc_midday_18_20.to_csv( loc_ftr + "\\auc_mday_trn.csv", index = False )
auc_midday_21_23.to_csv( loc_ftr + "\\auc_mday_tst.csv", index = False )

del auc_18_20['auc_11_15']
del auc_21_23['auc_11_15']

## <span style="color:green"> **Recibio subasta entre 16 hs y 20 hs (Tarde)** </span> 

#### <span style="color:Orange"> **One-hot Encoding** </span> 

In [13]:
auc_18_20['auc_16_20'] = ( pd.to_datetime( auc_18_20['date'] ).dt.hour < 21) & ( pd.to_datetime( auc_18_20['date'] ).dt.hour > 15)
auc_21_23['auc_16_20'] = ( pd.to_datetime( auc_21_23['date'] ).dt.hour < 21) & ( pd.to_datetime( auc_21_23['date'] ).dt.hour > 15)

auc_after_18_20 = rh_trn.copy()
auc_after_21_23 = rh_tst.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_16_20':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_16_20':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_after_18_20 = auc_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_after_21_23 = auc_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_after_18_20["auc_16_20"] =  (auc_after_18_20["auc_16_20"] > 1).astype('int8')
auc_after_21_23["auc_16_20"] =  (auc_after_21_23["auc_16_20"] > 1).astype('int8')

auc_after_18_20.to_csv( loc_ftr + "\\auc_aftr_trn.csv", index = False )
auc_after_21_23.to_csv( loc_ftr + "\\auc_aftr_tst.csv", index = False )

del auc_18_20['auc_16_20']
del auc_21_23['auc_16_20']